In [ ]:
import pandas as pd #ler os dados
import matplotlib.pyplot as plt #exibir gráfico

from sklearn import preprocessing #deixar os dados no intervalo [0,1]
from sklearn.model_selection import train_test_split #separar um conjunto de dados em 4 partes,sendo 2 os dados e 2 as classes relativas aos dados
from sklearn.decomposition import PCA

from keras import optimizers #importar otimizador usado no modelo (adam)
from keras.models import Sequential
from keras.utils import to_categorical #converter um array de números inteiros em uma matriz binária
from keras.layers import Dense, BatchNormalization

lendo os dados de treino e teste

In [ ]:
pd.options.display.max_columns = None

treino = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
teste = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')

convertendo a classe de categórica para numérica

In [ ]:
print('Antes')
print(treino['target'])

classes = {classe_orig:indice for indice, classe_orig in enumerate(sorted(treino['target'].unique()))}
treino['target'] = treino['target'].map(classes)

print('\nDepois')
print(treino['target'])

removendo coluna 'id' dos conjuntos de treino e teste

In [ ]:
treino.drop(['id'], inplace=True, axis=1)
teste.drop(['id'], inplace=True, axis=1)

removendo a coluna classe do conjunto de treino e armazenando seus dados em 'target'

In [ ]:
target = treino['target']
treino.drop(['target'], inplace=True, axis=1)

verificando se há valores omissos entre os dados

In [ ]:
print(treino.isnull().values.any())
print(target.isnull().values.any())
print(teste.isnull().values.any())

In [ ]:
pca = PCA(n_components = 53)
treino = pca.fit_transform(treino)
teste = pca.transform(teste)

pré-processamento dos dados, os dados são normalizados para estarem sempre no intervalo [0,1]

a parte comentada é minha implementação manual, mas ela demora muito para executar então substituí por uma função pronta

In [ ]:
# atributos = [[] for y in range(treino[0].size)]
# for i in treino:
#     for j in range(0, i.size):
#         atributos[j].append(i[j]) #colocando valores da coluna atual no array atributos
# for i in treino:
#     for j in range(0, i.size):
#         menor = min(atributos[j]) #menor valor da col atual
#         maior = max(atributos[j]) #maior valor da col atual
#         i[j] = (i[j]-menor)/(maior-menor) #novo valor no intervalo [0,1]

treino = preprocessing.normalize(treino)
teste = preprocessing.normalize(teste)

convertendo o array com as classes (numeros inteiros) em uma matriz binária de classes

In [ ]:
target = target.values
print('Antes')
print(target)

target = to_categorical(target)
print('Depois')
print(target)

separando o conjunto de treino em dois conjuntos: treino (70%) e validação (30%)

In [ ]:
x_treino, x_val, y_treino, y_val = train_test_split(treino, target, test_size=0.3, stratify=target)

definindo parametros para a rede neural (taxa de aprendizagem, epocas e otimizador)

In [ ]:
taxa_apr = 0.01
epocas = 10
otimizador = optimizers.Adam(lr=taxa_apr)

criando modelo e adicionando camadas escondidas a ele

a primeira camada recebe como parâmetro a quantidade de colunas do conjunto que será inserido nela

a última camada recebe como parâmetro a quantidade de colunas da saída

In [ ]:
model = Sequential()

model.add(Dense(40, input_dim = 53, activation='sigmoid'))
model.add(BatchNormalization())

model.add(Dense(20, activation='sigmoid'))
model.add(BatchNormalization())

# model.add(Dense(9))
# model.add(BatchNormalization())

model.add(Dense(9, activation='softmax'))

compilando o modelo criado e exibindo sua configuração

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=otimizador, metrics=['accuracy'])
print(model.summary())

treinando o modelo e armazenando dados sobre o treino em 'dados'

In [ ]:
dados = model.fit(
    x_treino,
    y_treino,
    batch_size=100,
    validation_data=(x_val, y_val),
    epochs=epocas,
    verbose = 1
)

verificando taxa de acerto para o conjunto de validação

In [ ]:
acerto_val = dados.history.get('val_accuracy')[-1]
print('Taxa de acerto na validação:', acerto_val*100, '%')

avaliando taxas de acerto do modelo para treino e validação

In [ ]:
plt.plot(dados.history['accuracy'], 'r-', label = 'accuracy', linewidth=2)
plt.plot(dados.history['val_accuracy'], 'c-',  label='accuracy' )
plt.ylabel('Acerto')
plt.xlabel('Época')
plt.legend(['Acerto - Treino', 'Acerto - Validação'], loc='upper right')
plt.show

saída: documento de exemplo de saída fornecido pela competição com os valores para cada classe substituídos pela classificação obtida ao executar o modelo no conjunto de teste

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4','Class_5','Class_6', 'Class_7', 'Class_8','Class_9']] = model.predict(teste)

print(sample_submission)

sample_submission.to_csv(f'submission.csv',index=False)